In [1]:
import os
import json
import base64
import numpy as np
import pandas as pd
import geopandas as gpd
from typing import Optional
from pyiceberg.expressions import And, GreaterThanOrEqual, LessThanOrEqual
from pyiceberg.catalog import load_catalog

In [2]:
import numpy as np
import geopandas as gpd
from scipy.spatial import cKDTree

def search_spatial_candidates(
    reference_gdf: gpd.GeoDataFrame,
    compared_gdf: gpd.GeoDataFrame,
    k: int = 100,
    max_dist: float = 1000, 
    id_col: str = "id",
    crs_for_distance: int = 3857,
):
    """
    Attach k nearest compared POI ids & distances to reference_gdf.

    Returns
    -------
    GeoDataFrame with two new columns:
    - cand_ids   : list of compared ids
    - cand_dist_m: list of distances (meters)
    """

    ref_proj = reference_gdf.to_crs(crs_for_distance)
    cmp_proj = compared_gdf.to_crs(crs_for_distance)

    ref_xy = np.column_stack([ref_proj.geometry.x, ref_proj.geometry.y])
    cmp_xy = np.column_stack([cmp_proj.geometry.x, cmp_proj.geometry.y])

    tree = cKDTree(cmp_xy)
    k_eff = min(k, len(compared_gdf))

    dist, idx = tree.query(ref_xy, k=k_eff)

    if k_eff == 1:
        dist = dist.reshape(-1, 1)
        idx = idx.reshape(-1, 1)

    cmp_ids = compared_gdf[id_col].to_numpy()

    cand_ids = []
    cand_dists = []

    for row_idx, row_dist in zip(idx, dist):
        ids = []
        dists = []

        for j, d in zip(row_idx, row_dist):
            if d <= max_dist:
                ids.append(cmp_ids[j])
                dists.append(d)

        cand_ids.append(ids)
        cand_dists.append(dists)

    result = reference_gdf.copy()
    result["cand_ids"] = cand_ids
    result["cand_dist_m"] = cand_dists

    return result

In [3]:
FOOD_WORDS = {
    "RESTAURANT","RESTURANT","RESTARAUNT",
    "CAFE","CAFÉ","COFFEE","BAR","BISTRO","GRILL",
    "KITCHEN","DINER","EATERY","STEAKHOUSE",
    "SEAFOOD","BUFFET","BBQ","PIZZA","SUSHI","RAMEN",
    "NOODLE","NOODLES","BURGER","BURGERS","TACO","TACOS",
    "CHICKEN","WINGS","BAKERY","DELI","DELICATESSEN",
    "COURT","FOOD","EXPRESS","HOUSE","SHOP"
}

PLACE_WORDS = {
    "HALL","CENTER","CENTRE","PLAZA","MARKET","MALL",
    "GARDEN","GARDENS","PARK","SQUARE","TOWER","TOWERS",
    "STATION","TERMINAL","BUILDING","GALLERY","THEATER","SCHOOL","COURT","INN",
    "HOTEL","MOTEL","INN","SUITES","SUITE",
    "SPA","SALON","STUDIO","CENTER","CENTRE",
    "CLUB","LOUNGE","STATION","STOP"
}

LEGAL_WORDS = {
    "LLC","INC","CORP","CORPORATION","CO","COMPANY",
    "LTD","LIMITED","GROUP","HOLDINGS","OFFICE"
}

GRAMMAR = {
    "THE","OF","AT","AND","FOR","IN","ON","BY","WITH","TO","FROM"
}

NON_PRIMARY_TOKENS = FOOD_WORDS | PLACE_WORDS | LEGAL_WORDS | GRAMMAR

In [4]:
from rapidfuzz import process, fuzz
import pandas as pd
import re
import unicodedata


def clean_name(s):
    if not isinstance(s, str):
        return ""

    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c)) # 1. unicode normalize (remove accents)
    s = s.upper() # 2. uppercase
    s = re.sub(r"\([^)]*\)", "", s) 
    s = re.sub(r"\b'S\b", "", s) # new change
    s = re.sub(r"\bS\b", "", s) # new change
    s = s.encode("ascii", errors="ignore").decode() # 4. remove emoji / non ascii
    s = re.sub(r"[^\w\s]", " ", s) # 5. replace special chars with space
    s = re.sub(r"\s+", " ", s) # 6. collapse spaces

    return s.strip()

def extract_prinmary_str(name):

    tokens = name.split()
    core = [t for t in tokens if t not in NON_PRIMARY_TOKENS]
    if len(core) == 1 and len(core[0]) < 3:
        return name
    if core:
        return " ".join(core)
    return name

def match_by_name(
    reference_gdf: gpd.GeoDataFrame,
    compared_gdf: gpd.GeoDataFrame,
    re_name_col: str = "name",
    comp_name_col: str = "name",
    comp_id: str = "id",
    comp_id_col: str="cat_main",
    threshold: int = 80,
):
    """
    Perform WRatio name matching within spatial candidates.

    Returns
    -------
    GeoDataFrame with:
    - matched_id_name
    - name_score
    """

    # clean names for matching
    id_to_name_clean = compared_gdf.set_index(comp_id)[comp_name_col].apply(clean_name).apply(extract_prinmary_str).to_dict()
    # raw names for storage
    id_to_name_raw = compared_gdf.set_index(comp_id)[comp_name_col].to_dict()
    # raw compared df category
    id_to_cat = compared_gdf.set_index(comp_id)[comp_id_col].to_dict()

    matched_ids = []
    scores = []
    loc_dists = []
    matched_names = []
    matched_cats = []

    for _, row in reference_gdf.iterrows():
        query = extract_prinmary_str(clean_name(row.get(re_name_col)))

        if not isinstance(query, str) or not row["cand_ids"]:
            matched_ids.append(pd.NA)
            scores.append(pd.NA)
            loc_dists.append(pd.NA)
            matched_names.append(pd.NA)
            matched_cats.append(pd.NA)
            continue

        cand_names = [id_to_name_clean.get(cid, "") for cid in row["cand_ids"]]

        top_matches = process.extract(
            query,
            cand_names,
            scorer=fuzz.WRatio,
            limit=5
        )

        best_score = -1
        best_pos = None

        for name, wr, pos in top_matches:

            pr = fuzz.partial_ratio(query, name)
            ts = fuzz.token_sort_ratio(query, name)
            tset = fuzz.token_set_ratio(query, name)

            combined = max(wr, pr, ts, tset)

            if combined > best_score:
                best_score = combined
                best_pos = pos

        score = best_score

        if score >= threshold:
            matched_ids.append(row["cand_ids"][best_pos])
            scores.append(score)
            loc_dists.append(row["cand_dist_m"][best_pos])
            matched_names.append(id_to_name_raw.get(row["cand_ids"][best_pos]))
            matched_cats.append(id_to_cat.get(row["cand_ids"][best_pos]))
        else:
            matched_ids.append(pd.NA)
            scores.append(score)
            loc_dists.append(pd.NA)
            matched_names.append(pd.NA)
            matched_cats.append(pd.NA)


    result = reference_gdf.copy()
    result["matched_id"] = matched_ids
    result["name_score"] = scores
    result["location_distance"] = loc_dists
    result["matched_name"] = matched_names
    result["matched_cat_main"] = matched_cats

    return result

In [5]:
import pandas as pd
import geopandas as gpd
from rapidfuzz import fuzz
import re
import unicodedata

def clean_name(s):
    if not isinstance(s, str):
        return ""

    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c)) # 1. unicode normalize (remove accents)
    s = s.upper() # 2. uppercase
    s = re.sub(r"\([^)]*\)", "", s) 
    s = s.encode("ascii", errors="ignore").decode() # 4. remove emoji / non ascii
    s = re.sub(r"[^\w\s]", " ", s) # 5. replace special chars with space
    s = re.sub(r"\s+", " ", s) # 6. collapse spaces

    return s.strip()

def address_score_check(
    reference_gdf: gpd.GeoDataFrame,
    compared_gdf: gpd.GeoDataFrame,
    addr_col_ref: str = "addr_simple",
    addr_col_cmp: str = "address",
    matched_id_col: str = "matched_id",
    id_col: str = "id",
    out_col: str = "address_score",
    out_addr_col: str = "matched_address", 
):
    """
    Compute address similarity score (0-100) for already-matched rows.

    Only rows with non-null `matched_id_col` will be scored.
    Others will have NaN.

    Returns
    -------
    GeoDataFrame with a new column `out_col`.
    """

    # map: compared id -> compared address
    id_to_addr_clean = compared_gdf.set_index(id_col)[addr_col_cmp].apply(clean_name).to_dict()
    id_to_addr_raw = compared_gdf.set_index(id_col)[addr_col_cmp].to_dict()

    scores = []
    matched_addrs = []

    for _, row in reference_gdf.iterrows():
        matched_id = row.get(matched_id_col)

        # no matched id -> no score
        if pd.isna(matched_id):
            scores.append(pd.NA)
            matched_addrs.append(pd.NA)
            continue

        addr_ref = clean_name(row.get(addr_col_ref))
        addr_cmp = id_to_addr_clean.get(matched_id)

        if isinstance(addr_cmp, str) and addr_cmp.strip():
            matched_addrs.append(id_to_addr_raw.get(matched_id))
        else:
            matched_addrs.append(pd.NA)

        # missing address on either side -> no score
        if not isinstance(addr_ref, str) or not addr_ref.strip():
            scores.append(pd.NA)
            continue
        if not isinstance(addr_cmp, str) or not addr_cmp.strip():
            scores.append(pd.NA)
            continue

        wr = fuzz.WRatio(addr_ref, addr_cmp)
        pr = fuzz.partial_ratio(addr_ref, addr_cmp)
        ts = fuzz.token_sort_ratio(addr_ref, addr_cmp)
        tset = fuzz.token_set_ratio(addr_ref, addr_cmp)

        scores.append(int(max(wr, pr, ts, tset)))

    result = reference_gdf.copy()
    result[out_col] = scores
    result[out_addr_col] = matched_addrs
    return result

In [6]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

def calculate_similarity_check(
    df, 
    cat_col_ref: str = "primary_type", 
    cat_col_cmp: str = "matched_cat_main", 
    id_col: str = "matched_id", 
    result_col: str =  "category_sim",
):

    # 1. Setup Device
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    
    # 2. Primary Gatekeeper: matched_id must be present
    mask = df[id_col].notna() & (df[id_col].astype(str).str.strip() != "")
    
    # 3. Create a helper to handle potential Nulls in text columns
    temp_df = df[mask].copy()
    
    # Identify where text is actually missing within the masked rows
    text_missing_mask = temp_df[cat_col_ref].isna() | temp_df[cat_col_cmp].isna()
    
    # Fill NaNs with empty strings just for the encoding step
    texts_1 = temp_df[cat_col_ref].fillna("").astype(str).tolist()
    texts_2 = temp_df[cat_col_cmp].fillna("").astype(str).tolist()

    print(f"Encoding {len(temp_df)} rows...")

    # 4. Batch Encoding
    emb1 = model.encode(texts_1, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
    emb2 = model.encode(texts_2, batch_size=256, show_progress_bar=True, convert_to_tensor=True)

    # 5. Calculate Similarity
    scores = torch.nn.functional.cosine_similarity(emb1, emb2, dim=1).cpu().numpy()
    
    # 6. Apply NaN to rows where text was missing
    # Even if we encoded an empty string, the result isn't "real" if data was missing
    scores[text_missing_mask.values] = np.nan

    # 7. Map back to original dataframe
    df[result_col] = np.nan
    df.loc[mask, result_col] = scores
    
    return df

In [7]:
bsop_gplc = gpd.read_file('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/BSOP_MSA/bspo_google_place_5000/google_placescat_5000_clean.geojson')
google_naics_mapping = pd.read_csv('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/mapping_google_naics.csv')
bsop_gplc = bsop_gplc.merge(google_naics_mapping[['SubCategory','naics_code','naics_definition']], left_on = 'primary_type', right_on='SubCategory', how="left")
bsop_gplc['addr_simple'] = bsop_gplc['address'].str.split(',', n=1).str[0]
bsop_gplc = bsop_gplc.drop(columns=['SubCategory'])
bsop_gplc

,circle_id,id,name,address,primary_type,lat,lon,primary_cat,geometry,naics_code,naics_definition,addr_simple
0,0,ChIJwTqOxGO1kVQR3ffcwUqPDTc,Safeway Fuel Station,"23961 WA-3, Belfair, WA 98528, USA",gas_station,47.453613,-122.824365,automotive,POINT (-122.82436 47.45361),44.0,Retail Trade,23961 WA-3
1,0,ChIJpZyerHu1kVQRAc6e7nFRTOA,Local Wrench,"23530 WA-3 suite a, Belfair, WA 98528, USA",car_repair,47.447925,-122.828294,automotive,POINT (-122.82829 47.44793),81.0,Other Services (except Public Administration),23530 WA-3 suite a
2,0,ChIJ8SIqjWO1kVQR2blnwrvt0C4,Valvoline Instant Oil Change,"23970 N WA-3, Belfair, WA 98528, USA",car_repair,47.453070,-122.822240,automotive,POINT (-122.82224 47.45307),81.0,Other Services (except Public Administration),23970 N WA-3
3,0,ChIJ4ZaLfGO1kVQRKZIPKm91JmA,Chevron Belfair,"23880 WA-3, Belfair, WA 98528, USA",gas_station,47.452338,-122.824376,automotive,POINT (-122.82438 47.45234),44.0,Retail Trade,23880 WA-3
4,0,ChIJOYTHKWO1kVQRRXwvopTyBVw,QFC Fuel Center,"201 WA-300, Belfair, WA 98528, USA",gas_station,47.453683,-122.827552,automotive,POINT (-122.82755 47.45368),44.0,Retail Trade,201 WA-300
...,...,...,...,...,...,...,...,...,...,...,...,...
4256,42,ChIJD6bA4-Xgj1QRg1RuvTgcXe0,Paradise Bay Rd & S Bay Lane,"Port Ludlow, WA 98365, USA",bus_stop,47.912121,-122.692577,transportation,POINT (-122.69258 47.91212),48.0,Transportation and Warehousing,Port Ludlow
4257,42,ChIJLah42szmj1QRZpfU3Vo2xTY,Oak Bay Rd & Opp Olympus Blvd,"Washington 98365, USA",bus_stop,47.941977,-122.688553,transportation,POINT (-122.68855 47.94198),48.0,Transportation and Warehousing,Washington 98365
4258,44,ChIJ0Y_p64MCkFQRyYXYqJflZI8,Scatchet Head Rd at Bailey Rd (SB),"Washington 98236, USA",bus_stop,47.936720,-122.410862,transportation,POINT (-122.41086 47.93672),48.0,Transportation and Warehousing,Washington 98236
4259,44,ChIJKW2_64MCkFQR4vXk6q4IizQ,Scatchet Head Rd at Bailey Rd (NB),"Washington 98236, USA",bus_stop,47.936706,-122.410775,transportation,POINT (-122.41078 47.93671),48.0,Transportation and Warehousing,Washington 98236


In [8]:
import json
import pandas as pd

def safe_json_load(x):
    try:
        return json.loads(x) if isinstance(x, str) else x
    except:
        return None

def extract_categories(x):
    data = safe_json_load(x)
    if isinstance(data, dict):
        primary = data.get('primary')
        alternate_list = data.get('alternate', [])
        alternate_str = ", ".join(alternate_list) if alternate_list else None
        return primary, alternate_str
    return None, None

def extract_name(x):
    data = safe_json_load(x)
    if isinstance(data, dict):
        return data.get('primary')
    return None

def extract_address(x):
    data = safe_json_load(x)
    if isinstance(data, list) and len(data) > 0:
        return data[0].get('freeform')
    return None

def extract_sources(x):
    data = safe_json_load(x)
    if isinstance(data, list) and len(data) > 0:
        first = data[0] 
        return first.get('dataset'), first.get('record_id'), first.get('update_time')
    return None, None, None


bspo_ove = pd.read_csv('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/BSOP_MSA/bspo msa/bspo_ove.csv')

bspo_ove[['cat_main', 'cat_alt']] = bspo_ove['categories'].apply(
    lambda x: pd.Series(extract_categories(x))
)

bspo_ove['name'] = bspo_ove['names'].apply(extract_name)
bspo_ove['address'] = bspo_ove['addresses'].apply(extract_address)
bspo_ove[['src_dataset', 'src_record_id', 'src_update_time']] = bspo_ove['sources'].apply(lambda x: pd.Series(extract_sources(x)))


bspo_ove['WKB'] = bspo_ove['geometry'].apply(lambda x: x[2:])
geometry = gpd.GeoSeries.from_wkb(bspo_ove['WKB'], crs=4326)

bspo_ove = gpd.GeoDataFrame(bspo_ove, geometry=geometry)
bspo_ove = bspo_ove[['id','name','address','cat_main','cat_alt','confidence','operating_status','version','src_dataset','src_update_time','geometry']]
bspo_ove

,id,name,address,cat_main,cat_alt,confidence,operating_status,version,src_dataset,src_update_time,geometry
0,d44807ab-9820-427c-84b1-eac2e1c9cb1b,"Lake Leland Quilcene, Wa",None,lake,"active_life, sports_club_and_league",0.323479,open,2,meta,2025-12-01T08:00:00.000Z,POINT (-122.73965 47.96577)
1,77c81091-7d93-476f-9298-9f68fde97b0f,Kodama Farm and Food Forest,42 Tall Tree Ln,farm,"livestock_breeder, urban_farm",0.950063,open,7,meta,2025-12-01T08:00:00.000Z,POINT (-122.74099 47.96374)
2,3aeb2bf6-01e3-4263-a66b-a4ca82fef297,Egg and I Fuchsias,2027 Egg and I Rd,farm,"breakfast_and_brunch_restaurant, cafe",0.917024,open,5,meta,2025-12-01T08:00:00.000Z,POINT (-122.75817 47.95798)
3,3da06df0-7ef2-455f-9603-e58400207494,Phillips Painting,1691 Egg And I Rd,painting,None,0.947100,open,4,Microsoft,2025-05-29T06:47:15.977Z,POINT (-122.76448 47.95435)
4,bb303c91-e532-4f97-ba5b-406372fdee4d,Harmonys Way Farm,1460 Egg and I Rd,farm,"vitamins_and_supplements, insurance_agency",0.639236,open,4,meta,2025-12-01T08:00:00.000Z,POINT (-122.77004 47.95224)
...,...,...,...,...,...,...,...,...,...,...,...
12439,fc313c3b-949c-45f3-988b-59a40a2c236a,Mount Walker,Mount Walker,landmark_and_historical_building,"park, hiking_trail",0.639236,open,1,meta,2025-12-01T08:00:00.000Z,POINT (-122.90684 47.78887)
12440,060bb2a5-2553-4745-913c-09435773e188,Devils Lake,None,lake,"attractions_and_activities, landmark_and_histo...",0.950063,open,5,meta,2025-12-01T08:00:00.000Z,POINT (-122.88288 47.79234)
12441,9daec706-a795-443e-ad44-d6cca11badbf,Marmot Pass,None,hiking_trail,sporting_goods,0.639236,open,5,meta,2025-12-01T08:00:00.000Z,POINT (-122.91249 47.81141)
12442,5b80ea90-d852-4a48-b80b-09e0142ed578,Quilcene National Fish Hatchery,281 Fish Hatchery Rd,environmental_conservation_organization,"public_and_government_association, public_serv...",0.882999,open,5,meta,2025-12-01T08:00:00.000Z,POINT (-122.91698 47.80821)


In [ ]:
# bspo_ove.to_file('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/BSOP_MSA/bspo msa/bspo_ove.geojson', driver='GeoJSON')

In [13]:
bsop_gplc_ove = search_spatial_candidates(reference_gdf=bsop_gplc, compared_gdf=bspo_ove, k=100)

In [16]:
bsop_gplc_ove = match_by_name(reference_gdf=bsop_gplc_ove, compared_gdf=bspo_ove, re_name_col = "name", comp_name_col = "name", threshold=80)

In [17]:
bsop_gplc_ove = address_score_check(reference_gdf=bsop_gplc_ove, compared_gdf=bspo_ove)

In [19]:
bsop_gplc_ove = calculate_similarity_check(bsop_gplc_ove, cat_col_ref = "primary_type", cat_col_cmp = "matched_cat_main", id_col = "matched_id", result_col =  "category_sim")

Encoding 2748 rows...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
# transfer the X and Y into float type and deal with the address score
cols_to_fix = ['name_score', 'location_distance', 'address_score', 'category_sim']
for col in cols_to_fix:
    bsop_gplc_ove[col] = pd.to_numeric(bsop_gplc_ove[col], errors='coerce')

In [24]:
df = bsop_gplc_ove[bsop_gplc_ove["matched_id"].notna()].copy()

N = 2000

weights = df["primary_cat"].map(
    df["primary_cat"].value_counts(normalize=True)
)

df_sample = df.sample(
    n=N,
    weights=weights,
    random_state=42
)
df_sample[['id','name','addr_simple','naics_definition','matched_id','matched_name','matched_address','matched_cat_main','location_distance']].to_csv('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/BSOP_MSA/bspo_google_comparison/bsop_gplc_ove_2000_sample.csv', index=False)

In [25]:
bsop_gplc_ove['matched_id'].notna().sum() / len(bsop_gplc_ove)

0.6449190330908238

In [28]:
df_sample_check = pd.read_csv('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/BSOP_MSA/bspo_google_comparison/bsop_gplc_ove_2000_sample.csv')
df_sample_check = df_sample_check.drop(columns='location_distance')
df_sample_check = df_sample_check.merge(bsop_gplc_ove[['id','name_score','location_distance','address_score','category_sim']], left_on="id", right_on="id", how="left")

In [29]:
df_sample_check['is_true'].value_counts()

is_true
1    1706
0     294
Name: count, dtype: int64

In [30]:
df = df_sample_check.copy()

In [31]:
from sklearn.preprocessing import StandardScaler

X = df[['name_score', 'location_distance', 'address_score', 'category_sim']]
y = df['is_true']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.25,
    random_state=42,
    stratify=y  # keep the same proportion of True vs False in training set and test set
)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

xgb_clf = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    enable_categorical=False,
    eval_metric='auc',
    random_state=42
)

xgb_clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False  
)

xgb_y_pred = xgb_clf.predict(X_test)
xgb_y_prob = xgb_clf.predict_proba(X_test)[:, 1]

xgb_cr = classification_report(y_test, xgb_y_pred)
xgb_auc = roc_auc_score(y_test, xgb_y_prob)

print(xgb_cr)
print("XGBoost AUC:", xgb_auc)

In [ ]:
mask = bsop_gplc_ove['matched_id'].notnull()
df_pred = bsop_gplc_ove.loc[mask].copy()

feature_cols = ['name_score', 'location_distance', 'address_score', 'category_sim']

X_new = scaler.transform(df_pred[feature_cols])
df_pred['true_match_prob'] = xgb_clf.predict_proba(X_new)[:, 1]
df_pred['is_true_match'] = df_pred['true_match_prob'] >= 0.5

bsop_gplc_ove.loc[mask, 'is_true_match'] = df_pred['is_true_match']
bsop_gplc_ove.loc[mask, 'true_match_prob'] = df_pred['true_match_prob']

In [35]:
bsop_gplc_ove['is_true_match'].value_counts()

is_true_match
True     2339
False     409
Name: count, dtype: int64

In [38]:
bsop_gplc_ove.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4261 entries, 0 to 4260
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   circle_id          4261 non-null   int64   
 1   id                 4261 non-null   object  
 2   name               4261 non-null   object  
 3   address            4261 non-null   object  
 4   primary_type       4261 non-null   object  
 5   lat                4261 non-null   float64 
 6   lon                4261 non-null   float64 
 7   primary_cat        4261 non-null   object  
 8   geometry           4261 non-null   geometry
 9   naics_code         4257 non-null   float64 
 10  naics_definition   4257 non-null   object  
 11  addr_simple        4261 non-null   object  
 12  cand_ids           4261 non-null   object  
 13  cand_dist_m        4261 non-null   object  
 14  matched_id         2748 non-null   object  
 15  name_score         3704 non-null   float64 
 16

In [40]:
bsop_gplc_ove.drop(columns=['cand_ids','cand_dist_m']).to_file('/Users/houpuli/Redlining Lab Dropbox/HOUPU LI/POI research/BSOP_MSA/bspo_google_comparison/bsop_gplc_ove.geojson', driver='GeoJSON')